In [1]:
import numpy as np

In [1]:
class Golub_Reisch_Kahan_SVD_Algorithm:
    def __init__(self,matrix):
        self.matrix = matrix
        self.num_rows, self.num_columns = self.GET_MATRIX_ROWS_COLUMNS(matrix) # Use a function to get rows and matrices
        self.computations = {} # Store Bidiag computations in a dict
        self.U_matrix = []
        self.V_matrix = []
        
    def GET_MATRIX_ROWS_COLUMNS(self, matrix):
        # Number of rows and columns in the matrix
        num_rows, num_cols = matrix.shape
        return num_rows, num_cols
    
    def SET_LOWVAL_ZERO(self, matrix):
        # This is for turning values into zeroes once they reach a certain threshold
        low_values_indices = abs(matrix) < 9e-15 
        matrix[low_values_indices] = 0
        return matrix
    
    def STORE_BIDIAG_COMPUTATIONS(self, i, x_vector, w_vector, v_vector, matrix): 
        # Store computations for the loop
        self.computations[i] = {
            'x_vector': x_vector.tolist(),
            'w_vector': w_vector.tolist(),
            'v_vector': v_vector.tolist(),
            'resulting_matrix': matrix.tolist()
        }
        return None
    
    def Householder_Reflector(self, vector, i):
        # For computing x, v, and w vectors as well as the P matrix
        # We change the sign depending on the sign of first element of the x vector
        alpha = -np.sign(vector[i]) * np.linalg.norm(vector)  
        e_vector = np.zeros(len(vector))
        e_vector[i] = 1.0
        
        # We then calculate the v and w vector as well as the P matrix
        w_vector = (vector - alpha * e_vector)
        v_vector = w_vector / np.linalg.norm(w_vector)
        P_matrix = np.eye(len(vector)) - 2 * np.outer(v_vector, v_vector.T)
        
        return P_matrix, vector, w_vector, v_vector
    
    def Wilkinson_Shift(self, matrix):
        # Compute for the shift 
        matrix_2x2 = matrix[-2:, -2:]  # Bottom-most right of matrix A [(a,b),(b,c)]. We expect it to be symmetric, but non-symmetric should be fine
        a = matrix_2x2[0, 0]
        b = matrix_2x2[0, 1]
        c = matrix_2x2[1, 0]

        print("")
        print("\033[1mBottom-most right 2x2 of matrix A:\033[0m")
        print("")
        print(matrix_2x2)

        l = (a - c) / 2

        if l < 0:
            shift = c + (b * b) / (abs(l) + (l * l + b * b) ** 0.5) ** 2
        elif l > 0:
            shift = c - (b * b) / (abs(l) + (l * l + b * b) ** 0.5) ** 2
        else:
            shift = c + (b * b) / (abs(l) + (l * l + b * b) ** 0.5) ** 2

        shift = complex(shift, 0.0)

        print("")
        print("Computed shift: " + str(shift))
        print("")
        

        return shift
    
    def Givens_Rotation(self, B_matrix, k):
        num_rows, num_columns = B_matrix.shape
        U2_matrix = np.eye(num_rows)
        V2_matrix = np.eye(num_columns)
        
        for i in range(num_columns - 1, k, -1):
            # Compute Givens rotation parameters
            a = B_matrix[i-1, i-1]
            b = B_matrix[i-1, i]
            c = B_matrix[i, i]

            r = np.sqrt(a**2 + b**2)
            cos_theta = a / r
            sin_theta = -b / r

            # Update B matrix with Givens rotation
            G_matrix = np.array([[cos_theta, -sin_theta],
                          [sin_theta, cos_theta]])
            
            B_matrix[i-1:i+1, i-1:i+1] = G_matrix.T @ B_matrix[i-1:i+1, i-1:i+1] @ G_matrix

            # Update U2 and V2 matrices
            U2_matrix[:, i-1:i+1] = U2_matrix[:, i-1:i+1] @ G_matrix
            V2_matrix[:, i-1:i+1] = V2_matrix[:, i-1:i+1] @ G_matrix

        return B_matrix, U2_matrix, V2_matrix
    
    def Compute_SVD(self, tol=1e-8, max_iter=100):
        for i in range(max_iter):
            # Step 1: Bidiagonalization with Wilkinson Shift
            matrix = self.Golub_Kahan_Bidiagonalization_with_Wilkinson_Shift()

            # Step 2: Reduction to Diagonal Form with Givens Rotations
            B_matrix, U2_matrix, V2_matrix = self.Givens_Rotation(matrix, 0)

            # Update U and V
            self.U_matrix[-1] = self.U_matrix[-1] @ U2_matrix
            self.V_matrix[-1] = self.V_matrix[-1] @ V2_matrix
            
            # Print computations per iteration
            self.print_household_reflector_computations()            

            # Check for convergence
            if np.abs(B_matrix[self.num_rows - 2, self.num_columns - 1]) < tol:
                break

        # Extract singular values and vectors
        singular_values = np.diag(B_matrix)
        singular_vectors = self.U_matrix[-1]

        return singular_values, singular_vectors
    
    def find_smallest_p_largest_q(B_matrix, epsilon):
        num_rows, num_columns = B_matrix.shape
        p = 0
        q = num_columns - 1

        for i in range(num_columns - 1):
            condition = abs(B_matrix[i, i+1]) <= epsilon * (abs(B_matrix[i, i]) + abs(B_matrix[i+1, i+1]))

            if not condition:
                p = i
                break

        for i in range(num_columns - 1, 0, -1):
            condition = abs(B_matrix[i-1, i]) <= epsilon * (abs(B_matrix[i-1, i-1]) + abs(B_matrix[i, i]))

            if not condition:
                q = i
                break

        return p, q


        
    def Golub_Kahan_Bidiagonalization_with_Wilkinson_Shift(self):
        matrix = self.matrix
        # Initialize U and V as identity matrices
        U_matrix = np.eye(self.num_rows)
        V_matrix = np.eye(self.num_columns)
        
        # This algorithms will only run householder reflectors in the minimum no. of rows and columns
        # This is for cases of a non-square matrix A
        # Excess rows or columns will be left out
        if self.num_rows <= self.num_columns:
            num_iter = self.num_rows - 1
        else:
            num_iter = self.num_columns - 1
        
        for i in range(num_iter):
            # Performing Householder Reflectors column wise
            x_vector = np.zeros(len(matrix[:, i]))
            x_vector[i:] = matrix[i:, i]
            P_matrix, x_vector, w_vector, v_vector = self.Householder_Reflector(x_vector, i)

            # Apply Wilkinson shift
            shift = self.Wilkinson_Shift(matrix[i:, i:])
            matrix[i:, i:] -= shift * np.eye(len(x_vector))
            
            matrix = self.SET_LOWVAL_ZERO(P_matrix @ matrix)
            U_matrix = U_matrix @ P_matrix  # Update matrix U
            self.U_matrix.append(U_matrix)
            self.STORE_BIDIAG_COMPUTATIONS(i, x_vector, w_vector, v_vector, matrix)
            

            # Performing Householder Reflectors row wise
            x_vector = np.zeros(len(matrix[i, :]))
            x_vector[i+1:] = matrix[i, i+1:] 
            Q_matrix, x_vector, w_vector, v_vector  = self.Householder_Reflector(x_vector, i+1)
            
            # Apply Wilkinson shift
            shift = self.Wilkinson_Shift(matrix[:, i+1:])
            matrix[:, i+1:] -= shift * np.eye(len(x_vector))
            
            matrix = self.SET_LOWVAL_ZERO(matrix @ Q_matrix)
            V_matrix = Q_matrix @ V_matrix  # Update matrix V
            self.V_matrix.append(U_matrix)
            self.STORE_BIDIAG_COMPUTATIONS(i+1, x_vector, w_vector, v_vector, matrix)
            
        
        # Truncate the resulting matrix
        matrix = np.trunc(matrix)   
        # Run the print function that prints out all of the computations for Bidiagonalization
        print_golub_kahan = self.print_household_reflector_computations()
        return matrix
    
    def print_household_reflector_computations(self):

        # For printing computations of Bidiagonalization that is stored from the dictionary
        # Formatting options for np.array2string
        format_options = {
            'formatter': {'all': '{:.4f}'.format},  # Specific number of decimal places
            'suppress_small': True,  
            'separator': ', ',  
        }

        print("________SOLUTION_________")
        print("")
        print("")

        # For each item in the dictionary we print x vector, w vector, v vector, and the resulting matrix    
        for i in range(len(self.computations)):
            iteration_data = self.computations[i]
            print("\033[1mIteration \033[0m" + str(i+1) + ":")

            print("\033[1mx vector: \033[0m" + str(iteration_data['x_vector']))
            print("\033[1mw vector: \033[0m" + str(iteration_data['w_vector']))
            print("\033[1mv vector: \033[0m" + str(iteration_data['v_vector']))
            print("")

            matrix = np.array(iteration_data['resulting_matrix'])

            # Printing resulting matrix each iteration
            print("\033[1mB matrix: \033[0m")
            print("")
            print(np.array2string(matrix, **format_options))
            print("")
            print("")
            
        print("\033[1mU matrix:\033[0m")
        print(self.U_matrix[-1])
        print("")

        print("\033[1mVᵀ matrix:\033[0m")
        print((self.V_matrix[-1]).T)
        print("")
        
        # Reconstruct the matrix
        A_reconstructed = np.dot(self.U_matrix[-1], np.dot(np.diag(np.linalg.svd(matrix, full_matrices=False)[1]), (self.V_matrix[-1]).T))

        # Check if the reconstructed matrix is close to the original matrix
        print(matrix)
        print(A_reconstructed)
        
        norm_B = np.linalg.norm(matrix)
        norm_A_reconstructed = np.linalg.norm(A_reconstructed)

        print(f"Norm of Bidiagonalized: {norm_B}")
        print(f"Norm of A_reconstructed: {norm_A_reconstructed}")



        return None

    



In [2]:
#We use np.random to generate a random matrix
A_matrix = np.random.rand(4, 4)

print("A matrix =")
print("          ")
print(A_matrix)

Solution = Golub_Reisch_Kahan_SVD_Algorithm(A_matrix)
Solution.Golub_Kahan_Bidiagonalization()

NameError: name 'np' is not defined